In [2]:
import z3
from z3 import Optimize, Int, Real, Bool, If, ToReal, ToInt, sat

def _calculate_foreigner_tax_internal(
    basic_living_exp_per_person: int = 210000,           # 每人基本生活費用
    savings_investment_deduction_limit: int = 270000,      # 儲蓄投資扣除上限
    disability_deduction_per_person: int = 218000,         # 每位身心障礙者扣除額
    education_deduction_per_student: int = 25000,          # 每位學生教育扣除額
    long_term_care_deduction_per_person: int = 120000,     # 每人長期照護扣除額
    rent_deduction_limit: int = 180000,                    # 房屋租金特別扣除上限

    personal_exemption_under70: int = 97000,               # 70歲以下個人免稅額
    personal_exemption_over70: int  = 145500,              # 70歲以上個人免稅額

    standard_deduction_single: int  = 131000,              # 單身標準扣除額
    standard_deduction_married: int = 262000,              # 已婚標準扣除額
    salary_special_deduction_max: int = 218000,            # 薪資特別扣除額上限

    bracket1_upper: int = 590000,                          # 速算級距1上限
    bracket2_upper: int = 1330000,                         # 速算級距2上限
    bracket3_upper: int = 2660000,                         # 速算級距3上限
    bracket4_upper: int = 4980000,                         # 速算級距4上限

    bracket1_rate: float = 0.05,                           # 速算級距1稅率
    bracket2_rate: float = 0.12,                           # 速算級距2稅率
    bracket3_rate: float = 0.20,                           # 速算級距3稅率
    bracket4_rate: float = 0.30,                           # 速算級距4稅率
    bracket5_rate: float = 0.40,                           # 速算級距5稅率

    bracket1_sub: int = 0,                                 # 速算級距1扣減額
    bracket2_sub: int = 41300,                             # 速算級距2扣減額
    bracket3_sub: int = 147700,                            # 速算級距3扣減額
    bracket4_sub: int = 413700,                            # 速算級距4扣減額
    bracket5_sub: int = 911700,                            # 速算級距5扣減額

    # ===== 外僑申報資料 =====
    days_of_stay: int = 365,         # 外僑在台居留天數
    is_departure: bool = False,      # 是否提前離境
    is_married: bool = False,        # 是否已婚
    salary_self: int = 0,            # 本人薪資所得
    salary_spouse: int = 0,          # 配偶薪資所得
    salary_dep: int = 0,             # 受扶養親屬薪資所得
    interest_income: int = 0,        # 本人利息所得
    interest_spouse: int = 0,        # 配偶利息所得
    interest_dep: int = 0,           # 受扶養親屬利息所得
    other_income: int = 0,           # 本人其他所得
    other_income_spouse: int = 0,    # 配偶其他所得
    other_income_dep: int = 0,       # 受扶養親屬其他所得
    cnt_under_70: int = 0,           # 70歲以下人數
    cnt_over_70: int = 0,            # 70歲以上人數
    use_itemized: bool = False,      # 是否使用列舉扣除
    itemized_deduction: int = 0,     # 列舉扣除額
    property_loss_deduction: int = 0,# 財產損失扣除額
    disability_count: int = 0,       # 身心障礙者人數
    education_count: int = 0,        # 教育扣除適用人數
    education_fee: int = 0,          # 實際支付教育費用
    preschool_count: int = 0,        # 幼兒人數
    long_term_care_count: int = 0,   # 長期照護適用人數
    rent_deduction: int = 0,         # 房屋租金支出

    free_vars: list = None         # 指定哪些變數為自由變數（例如 ["salary_spouse", "interest_income", "interest_spouse", "property_loss_deduction"]）
):
    """
    計算外僑綜合所得稅（內部計算函式）
    除了原有計算邏輯外，若使用者指定了 free_vars（自由變數），則分兩階段：
      1. 先以所有參數固定求出初始稅額（未用 free_vars 優化）。
      2. 再釋放 free_vars 為自由變數，並反覆求解直到最終稅額無法進一步改善，
         同時列印每次迭代過程。
    傳回:
      (final_tax, final_vars, free_vars_diff)
        final_tax: 最終應納稅額
        final_vars: 各參數的最終值與型態
        free_vars_diff: free 變數從原始到最終的變化情形
    """

    # 先計算居留比例：若提前離境則依實際居留天數調整
    residency_ratio = 1
    if is_departure:
        residency_ratio = days_of_stay / 365

    # 調整後的扣除額參數
    adj_personal_exemption_under70 = int(personal_exemption_under70 * residency_ratio)
    adj_personal_exemption_over70  = int(personal_exemption_over70 * residency_ratio)
    adj_standard_deduction_single  = int(standard_deduction_single * residency_ratio)
    adj_standard_deduction_married = int(standard_deduction_married * residency_ratio)
    adj_basic_living_exp_per_person= int(basic_living_exp_per_person * residency_ratio)

    # ---------------------------
    # 1. 定義 Z3 變數（共用）
    # ---------------------------
    salary_self_z   = Int('salary_self_z')
    salary_spouse_z = Int('salary_spouse_z')
    salary_dep_z    = Int('salary_dep_z')
    interest_z      = Int('interest_z')
    other_income_z  = Int('other_income_z')
    itemized_ded_z  = Int('itemized_ded_z')
    prop_loss_ded_z = Int('prop_loss_ded_z')
    rent_ded_z      = Int('rent_ded_z')

    interest_spouse_z     = Int('interest_spouse_z')
    interest_dep_z        = Int('interest_dep_z')
    other_income_spouse_z = Int('other_income_spouse_z')
    other_income_dep_z    = Int('other_income_dep_z')
    stock_div_z           = Int('stock_div_z')   # 股票股利
    house_gain_z          = Int('house_gain_z')  # 房屋交易所得

    total_income_z  = Int('total_income_z')

    self_salary_special_deduction_z   = Int('self_salary_special_deduction_z')
    spouse_salary_special_deduction_z = Int('spouse_salary_special_deduction_z')
    dep_salary_special_ded_z          = Int('dep_salary_special_ded_z')

    self_salary_after_ded_z   = Int('self_salary_after_ded_z')
    spouse_salary_after_ded_z = Int('spouse_salary_after_ded_z')
    dep_salary_after_ded_z    = Int('dep_salary_after_ded_z')

    total_exemption_z    = Int('total_exemption_z')
    standard_deduction_z = Int('standard_deduction_z')
    chosen_deduction_z   = Int('chosen_deduction_z')

    savings_investment_deduction_z = Int('savings_investment_deduction_z')
    disability_deduction_z         = Int('disability_deduction_z')
    education_deduction_z          = Int('education_deduction_z')

    preschool_count_z     = Int('preschool_count_z')
    preschool_deduction_z = Int('preschool_deduction_z')

    long_term_care_deduction_z = Int('long_term_care_deduction_z')
    rent_deduction_z_lim       = Int('rent_deduction_z_lim')

    total_people_z = Int('total_people_z')
    basic_living_exp_total_z = Int('basic_living_exp_total_z')
    base_deductions_var_z = Int('base_deductions_var_z')
    basic_living_exp_diff_z = Int('basic_living_exp_diff_z')
    total_deduction_z = Int('total_deduction_z')
    net_taxable_income_z = Int('net_taxable_income_z')
    net_taxable_nonneg_z = Int('net_taxable_nonneg_z')

    rate_calc_r = Real('rate_calc_r')
    final_tax_z = Int('final_tax_z')

    days_of_stay_z      = Int('days_of_stay_z')
    is_departure_z      = Bool('is_departure_z')
    is_married_z        = Bool('is_married_z')
    cnt_under_70_z      = Int('cnt_under_70_z')
    cnt_over_70_z       = Int('cnt_over_70_z')
    use_itemized_z      = Bool('use_itemized_z')
    disability_count_z  = Int('disability_count_z')
    education_count_z   = Int('education_count_z')
    education_fee_z     = Int('education_fee_z')
    long_term_care_count_z = Int('long_term_care_count_z')

    # ---------------------------
    # 2. 建立「輸入值 -> Z3 變數」及範圍檢查的 mapping
    # ---------------------------
    # 若參數名稱在 free_vars 裡，僅加上基本檢查；否則以等式約束固定
    if free_vars is None:
        free_vars = []
    params = {
        "salary_self": (salary_self_z, salary_self, [lambda var: var >= 0]),
        "salary_spouse": (salary_spouse_z, salary_spouse, [lambda var: var >= 0]),
        "salary_dep": (salary_dep_z, salary_dep, [lambda var: var >= 0]),
        "interest_income": (interest_z, interest_income, [lambda var: var >= 0]),
        "other_income": (other_income_z, other_income, [lambda var: var >= 0]),
        "itemized_deduction": (itemized_ded_z, itemized_deduction, [lambda var: var >= 0]),
        "property_loss_deduction": (prop_loss_ded_z, property_loss_deduction, [lambda var: var >= 0]),
        "rent_deduction": (rent_ded_z, rent_deduction, [lambda var: var >= 0, lambda var: var <= 10000000]),
        "interest_spouse": (interest_spouse_z, interest_spouse, [lambda var: var >= 0]),
        "interest_dep": (interest_dep_z, interest_dep, [lambda var: var >= 0]),
        "other_income_spouse": (other_income_spouse_z, other_income_spouse, [lambda var: var >= 0]),
        "other_income_dep": (other_income_dep_z, other_income_dep, [lambda var: var >= 0]),
        "days_of_stay": (days_of_stay_z, days_of_stay, [lambda var: var >= 0]),
        "is_departure": (is_departure_z, is_departure, []),
        "is_married": (is_married_z, is_married, []),
        "cnt_under_70": (cnt_under_70_z, cnt_under_70, [lambda var: var >= 0]),
        "cnt_over_70": (cnt_over_70_z, cnt_over_70, [lambda var: var >= 0]),
        "use_itemized": (use_itemized_z, use_itemized, []),
        "disability_count": (disability_count_z, disability_count, [lambda var: var >= 0]),
        "education_count": (education_count_z, education_count, [lambda var: var >= 0]),
        "education_fee": (education_fee_z, education_fee, [lambda var: var >= 0]),
        "preschool_count": (preschool_count_z, preschool_count, [lambda var: var >= 0]),
        "long_term_care_count": (long_term_care_count_z, long_term_care_count, [lambda var: var >= 0]),
    }

    # ---------------------------
    # 3. 若有 free_vars，先以所有參數固定求解初始稅額
    # ---------------------------
    if free_vars:
        fixed_opt = Optimize()
        # 對所有參數皆加入等式約束（即把 free_vars 當作固定值）
        for param_name, (z3_var, value, constraints) in params.items():
            fixed_opt.add(z3_var == value)
            for cons in constraints:
                fixed_opt.add(cons(z3_var))
        # 加入後續「其他約束」（下同）
        fixed_opt.add(self_salary_special_deduction_z == If(salary_self_z >= salary_special_deduction_max, salary_special_deduction_max, salary_self_z))
        fixed_opt.add(spouse_salary_special_deduction_z == If(salary_spouse_z >= salary_special_deduction_max, salary_special_deduction_max, salary_spouse_z))
        fixed_opt.add(dep_salary_special_ded_z == If(salary_dep_z >= salary_special_deduction_max, salary_special_deduction_max, salary_dep_z))
        fixed_opt.add(self_salary_after_ded_z == salary_self_z - self_salary_special_deduction_z)
        fixed_opt.add(spouse_salary_after_ded_z == salary_spouse_z - spouse_salary_special_deduction_z)
        fixed_opt.add(dep_salary_after_ded_z == salary_dep_z - dep_salary_special_ded_z)
        fixed_opt.add(self_salary_after_ded_z >= 0)
        fixed_opt.add(spouse_salary_after_ded_z >= 0)
        fixed_opt.add(dep_salary_after_ded_z >= 0)
        fixed_opt.add(total_income_z == (self_salary_after_ded_z + spouse_salary_after_ded_z + dep_salary_after_ded_z
                                         + interest_z + interest_spouse_z + interest_dep_z
                                         + stock_div_z + house_gain_z
                                         + other_income_z + other_income_spouse_z + other_income_dep_z))
        fixed_opt.add(total_exemption_z == (cnt_under_70 * adj_personal_exemption_under70 + cnt_over_70 * adj_personal_exemption_over70))
        fixed_opt.add(standard_deduction_z == (adj_standard_deduction_married if is_married else adj_standard_deduction_single))
        fixed_opt.add(chosen_deduction_z == If(use_itemized, itemized_ded_z, standard_deduction_z))
        interest_plus_div_z = Int('interest_plus_div_z')
        fixed_opt.add(interest_plus_div_z == (interest_z + stock_div_z))
        fixed_opt.add(savings_investment_deduction_z == If(interest_plus_div_z <= savings_investment_deduction_limit, interest_plus_div_z, savings_investment_deduction_limit))
        fixed_opt.add(disability_deduction_z == disability_count * disability_deduction_per_person)
        fixed_opt.add(education_deduction_z == If(education_fee <= 0, 0, If(education_fee >= education_count * education_deduction_per_student, education_count * education_deduction_per_student, education_fee)))
        fixed_opt.add(preschool_deduction_z == If(preschool_count_z <= 0, 0, If(preschool_count_z == 1, 150000, 150000 + (preschool_count_z - 1) * 225000)))
        fixed_opt.add(long_term_care_deduction_z == long_term_care_count * long_term_care_deduction_per_person)
        fixed_opt.add(rent_deduction_z_lim == If(rent_ded_z >= rent_deduction_limit, rent_deduction_limit, rent_ded_z))
        fixed_opt.add(total_people_z == cnt_under_70 + cnt_over_70)
        fixed_opt.add(basic_living_exp_total_z == total_people_z * adj_basic_living_exp_per_person)
        fixed_opt.add(base_deductions_var_z == (total_exemption_z + chosen_deduction_z + savings_investment_deduction_z 
                                               + disability_deduction_z + education_deduction_z + preschool_deduction_z 
                                               + long_term_care_deduction_z + rent_deduction_z_lim))
        fixed_opt.add(basic_living_exp_diff_z == If(basic_living_exp_total_z > base_deductions_var_z, basic_living_exp_total_z - base_deductions_var_z, 0))
        fixed_opt.add(total_deduction_z == base_deductions_var_z + prop_loss_ded_z + basic_living_exp_diff_z)
        fixed_opt.add(net_taxable_income_z == total_income_z - total_deduction_z)
        fixed_opt.add(net_taxable_nonneg_z == If(net_taxable_income_z < 0, 0, net_taxable_income_z))
        x = ToReal(net_taxable_nonneg_z)
        fixed_opt.add(rate_calc_r == If(x <= bracket1_upper,
                                      (x * bracket1_rate) - bracket1_sub,
                                   If(x <= bracket2_upper,
                                      (x * bracket2_rate) - bracket2_sub,
                                   If(x <= bracket3_upper,
                                      (x * bracket3_rate) - bracket3_sub,
                                   If(x <= bracket4_upper,
                                      (x * bracket4_rate) - bracket4_sub,
                                      (x * bracket5_rate) - bracket5_sub)))))
        safe_tax_r = If(rate_calc_r < 0, 0, rate_calc_r)
        fixed_opt.add(final_tax_z == ToInt(safe_tax_r))
        if fixed_opt.check() != sat:
            print("初始條件無解")
            return None
        fixed_model = fixed_opt.model()
        initial_tax = fixed_model[final_tax_z].as_long()
        print("=== 優化過程開始 ===\n")
        print("初始稅額 (固定參數) : {}".format(initial_tax))
        print()
    else:
        # 沒有 free_vars 則初始稅額即為後續結果參數
        initial_tax = None

    # ---------------------------
    # 4. 建立 Optimize 物件進行 free_vars 優化
    # ---------------------------
    opt = Optimize()
    for param_name, (z3_var, value, constraints) in params.items():
        if param_name in free_vars:
            for cons in constraints:
                opt.add(cons(z3_var))
        else:
            opt.add(z3_var == value)
            for cons in constraints:
                opt.add(cons(z3_var))
    # 加入其他約束（與原邏輯保持一致）
    opt.add(self_salary_special_deduction_z == If(salary_self_z >= salary_special_deduction_max, salary_special_deduction_max, salary_self_z))
    opt.add(spouse_salary_special_deduction_z == If(salary_spouse_z >= salary_special_deduction_max, salary_special_deduction_max, salary_spouse_z))
    opt.add(dep_salary_special_ded_z == If(salary_dep_z >= salary_special_deduction_max, salary_special_deduction_max, salary_dep_z))
    opt.add(self_salary_after_ded_z == salary_self_z - self_salary_special_deduction_z)
    opt.add(spouse_salary_after_ded_z == salary_spouse_z - spouse_salary_special_deduction_z)
    opt.add(dep_salary_after_ded_z == salary_dep_z - dep_salary_special_ded_z)
    opt.add(self_salary_after_ded_z >= 0)
    opt.add(spouse_salary_after_ded_z >= 0)
    opt.add(dep_salary_after_ded_z >= 0)
    opt.add(total_income_z == (self_salary_after_ded_z + spouse_salary_after_ded_z + dep_salary_after_ded_z
                                + interest_z + interest_spouse_z + interest_dep_z
                                + stock_div_z + house_gain_z
                                + other_income_z + other_income_spouse_z + other_income_dep_z))
    opt.add(total_exemption_z == (cnt_under_70 * adj_personal_exemption_under70 + cnt_over_70 * adj_personal_exemption_over70))
    opt.add(standard_deduction_z == (adj_standard_deduction_married if is_married else adj_standard_deduction_single))
    opt.add(chosen_deduction_z == If(use_itemized, itemized_ded_z, standard_deduction_z))
    interest_plus_div_z = Int('interest_plus_div_z')
    opt.add(interest_plus_div_z == (interest_z + stock_div_z))
    opt.add(savings_investment_deduction_z == If(interest_plus_div_z <= savings_investment_deduction_limit, interest_plus_div_z, savings_investment_deduction_limit))
    opt.add(disability_deduction_z == disability_count * disability_deduction_per_person)
    opt.add(education_deduction_z == If(education_fee <= 0, 0, If(education_fee >= education_count * education_deduction_per_student, education_count * education_deduction_per_student, education_fee)))
    opt.add(preschool_deduction_z == If(preschool_count_z <= 0, 0, If(preschool_count_z == 1, 150000, 150000 + (preschool_count_z - 1) * 225000)))
    opt.add(long_term_care_deduction_z == long_term_care_count * long_term_care_deduction_per_person)
    opt.add(rent_deduction_z_lim == If(rent_ded_z >= rent_deduction_limit, rent_deduction_limit, rent_ded_z))
    opt.add(total_people_z == cnt_under_70 + cnt_over_70)
    opt.add(basic_living_exp_total_z == total_people_z * adj_basic_living_exp_per_person)
    opt.add(base_deductions_var_z == (total_exemption_z + chosen_deduction_z + savings_investment_deduction_z 
                                      + disability_deduction_z + education_deduction_z + preschool_deduction_z 
                                      + long_term_care_deduction_z + rent_deduction_z_lim))
    opt.add(basic_living_exp_diff_z == If(basic_living_exp_total_z > base_deductions_var_z, basic_living_exp_total_z - base_deductions_var_z, 0))
    opt.add(total_deduction_z == base_deductions_var_z + prop_loss_ded_z + basic_living_exp_diff_z)
    opt.add(net_taxable_income_z == total_income_z - total_deduction_z)
    opt.add(net_taxable_nonneg_z == If(net_taxable_income_z < 0, 0, net_taxable_income_z))
    x = ToReal(net_taxable_nonneg_z)
    opt.add(rate_calc_r == If(x <= bracket1_upper,
                              (x * bracket1_rate) - bracket1_sub,
                           If(x <= bracket2_upper,
                              (x * bracket2_rate) - bracket2_sub,
                           If(x <= bracket3_upper,
                              (x * bracket3_rate) - bracket3_sub,
                           If(x <= bracket4_upper,
                              (x * bracket4_rate) - bracket4_sub,
                              (x * bracket5_rate) - bracket5_sub)))))
    safe_tax_r = If(rate_calc_r < 0, 0, rate_calc_r)
    opt.add(final_tax_z == ToInt(safe_tax_r))
    # 若有 free_vars，要求優化解必須小於初始稅額
    if free_vars:
        opt.add(final_tax_z < initial_tax)

    # ---------------------------
    # 5. 進行反覆優化並列印每次過程
    # ---------------------------
    # 記錄 free 變數原始值（僅 free_vars）
    orig_free_values = {name: value for name, (_, value, _) in params.items() if name in free_vars}
    prev_free_values = orig_free_values.copy()

    if opt.check() != sat:
        print("初始條件無解")
        return None
    best_model = opt.model()
    best_tax = best_model[final_tax_z].as_long()
    # 如果沒有 free_vars，就以計算結果當初始稅額
    if not free_vars:
        print("初始稅額: {}".format(best_tax))
    else:
        print("初始優化解稅額: {}".format(best_tax))
    print()

    iteration = 0
    final_diff = {}  # 紀錄 free 變數從原始到最終的變化

    while True:
        iteration += 1
        if opt.check() != sat:
            print("----- 第 {} 次優化 -----".format(iteration))
            print("當前稅額: {}".format(best_tax))
            print("無法進一步優化，已達最佳解")
            break
        current_model = opt.model()
        current_tax = current_model[final_tax_z].as_long()
        print("----- 第 {} 次優化 -----".format(iteration))
        print("當前稅額: {}".format(current_tax))
        print("變更的變數:")
        iter_diff = {}
        for param_name in free_vars:
            z3_var, orig_value, _ = params[param_name]
            current_value = current_model[z3_var].as_long()
            prev_value = prev_free_values.get(param_name, orig_value)
            diff = current_value - prev_value
            if diff != 0:
                print("  {}: 原始 = {}, 優化後 = {}, 差異 = {}".format(param_name, prev_value, current_value, diff))
                iter_diff[param_name] = {"original": prev_value, "optimized": current_value, "difference": diff}
        if not iter_diff:
            print("  (無變更)")
        print("-------------------------")
        if current_tax < best_tax:
            best_tax = current_tax
            best_model = current_model
            for param_name in free_vars:
                z3_var, _, _ = params[param_name]
                prev_free_values[param_name] = current_model[z3_var].as_long()
            opt.add(final_tax_z < best_tax)
            final_diff = {name: {"original": orig_free_values[name],
                                 "optimized": current_model[params[name][0]].as_long(),
                                 "difference": current_model[params[name][0]].as_long() - orig_free_values[name]}
                          for name in free_vars}
        else:
            print("第 {} 次: unsat，無解，已是最佳解".format(iteration))
            break

    print("\n=== 優化結束，共 {} 次迭代，最終稅額 = {} ===".format(iteration, best_tax))
    print()

    # ---------------------------
    # 6. 組裝最終結果的字典
    # ---------------------------
    output_keys = ["salary_self", "salary_spouse", "salary_dep", "interest_income", "stock_dividend",
                   "house_transaction_gain", "other_income", "itemized_deduction", "property_loss_deduction",
                   "rent_deduction", "cnt_under_70", "cnt_over_70", "disability_count", "education_count",
                   "education_fee", "preschool_count", "long_term_care_count"]
    final_vars_dict = {}
    for key in output_keys:
        if key == "stock_dividend":
            value = best_model[stock_div_z].as_long() if best_model[stock_div_z] is not None else 0
            final_vars_dict[key] = {"value": value, "type": "free" if key in free_vars else "fixed"}
        elif key == "house_transaction_gain":
            value = best_model[house_gain_z].as_long() if best_model[house_gain_z] is not None else 0
            final_vars_dict[key] = {"value": value, "type": "free" if key in free_vars else "fixed"}
        elif key in params:
            z3_var, orig_value, _ = params[key]
            value = best_model[z3_var].as_long()
            final_vars_dict[key] = {"value": value, "type": "free" if key in free_vars else "fixed"}
    # 回傳 tuple: (最終稅額, 最終參數字典, free 變數變化字典)
    return best_tax, final_vars_dict, final_diff

def calculate_foreigner_income_tax(**kwargs):
    """
    計算外僑綜合所得稅（對外API函式）
    若淨所得超過 1,330,000（即進入20%以上級距），則長期照護及房租扣除不適用，
    此時會以調整後參數進行第二次試算。
    傳回:
      最終應納稅額
    """
    tax_val, final_vars, diff_details = _calculate_foreigner_tax_internal(**kwargs)
    # 此處也可將 final_vars 與 diff_details 印出或記錄下來
    if 'net_taxable_income' in final_vars and final_vars['net_taxable_income']['value'] >= 1330000:
        kwargs_modified = kwargs.copy()
        kwargs_modified['long_term_care_count'] = 0
        kwargs_modified['rent_deduction'] = 0
        tax_val2, _, _ = _calculate_foreigner_tax_internal(**kwargs_modified)
        return tax_val2
    else:
        return tax_val

In [7]:
tax = calculate_foreigner_income_tax(
        is_married=True,
        salary_self=3200000,
        salary_spouse=589914,
        salary_dep=119016, 
        interest_income=35504,
        interest_spouse=0,
        interest_dep=3000,
        other_income=152873,
        other_income_spouse=20000,
        other_income_dep=10000,
        cnt_under_70=1,
        cnt_over_70=3,
        use_itemized=True,
        itemized_deduction=441883,
        property_loss_deduction=181269,
        disability_count=1,
        education_count=0,
        education_fee=0,
        preschool_count=2,
        long_term_care_count=0,
        rent_deduction=88155,
        days_of_stay=320,
        is_departure=True,
        free_vars=['salary_spouse','salary_dep']
    )
print("113年度試算  =", tax)

=== 優化過程開始 ===

初始稅額 (固定參數) : 205851

初始優化解稅額: 0

----- 第 1 次優化 -----
當前稅額: 0
變更的變數:
  salary_spouse: 原始 = 589914, 優化後 = 217999, 差異 = -371915
  salary_dep: 原始 = 119016, 優化後 = 218001, 差異 = 98985
-------------------------
第 1 次: unsat，無解，已是最佳解

=== 優化結束，共 1 次迭代，最終稅額 = 0 ===

113年度試算  = 0
